# Week 2: Solving Systems of Linear Equations

## Gaussian Elimination Algorithm

In [1]:
import numpy as np

Gaussian elimination is a systematic approach to solves systems of linear equations. The algorithm consists of several steps:
1. Augment a matrix
2. Transform matrix into row-echelon form
3. Bask substitution (solve for variables from the last row upwards)
4. Combine functions into a single comprehensive function

### Step 1. Construct the Augmented Matrix

In [3]:
def augmented_matrix(A, B):
    """
    Horizonally stack matrices A and B
    """
    augmented_M = np.hstack((A, B))
    return augmented_M

### Step 2: Transform the matrix into row-echelon form
#### 2.1 - Theory 
A matrix in row echelon form follows:
- Rows consisting of entirely zeros positioned at the bottom
- Each non-zero row has its left-most non-zero coefficient (pivot) located to the right of that of the row above it. So all elements below the pivot within the same column are 0.

Example of matrix in row echelon form:
$$M =
\begin{bmatrix}
7 & 2 & 3 \\
0 & 9 & 4 \\
0 & 0 & 3 \\
\end{bmatrix}
$$

#### 2.2 - Algorithm Implementation
Consider matrix $M$ given by:

$$
M = 
\begin{bmatrix} 
* & * & * & \\
0 & \text{pivot} & * \\
0 & \text{value} & * 
\end{bmatrix}
$$

The asterisk (*) denotes any number. To nullify the last row (row $2$), we:

- Scale $R_1$ by the inverse of the pivot and get the updated matrix with the pivot for row $1$ set to $1$:

$$
\text{Row 1} \rightarrow \frac{1}{\text{pivot}} \cdot \text{Row } 
$$

$$
M = 
\begin{bmatrix} 
* & * & * & \\
0 & 1 & * \\
0 & \text{value} & * 
\end{bmatrix}
$$

- Then to eliminate the value below the pivot in row $1$, we subtract the row by the value timed the above row.

$$
\text{Row 2} \rightarrow \text{Row 2} - \text{value}\cdot \text{Row 1}
$$

$$
M = 
\begin{bmatrix} 
* & * & * & \\
0 & 1 & * \\
0 & 0 & * 
\end{bmatrix}
$$

In [4]:
def get_index_first_non_zero_value_from_column(M, column, starting_row):
    """
    Auxiliary function to get the index of the first non-zero value in a COLUMN of a given matrix
    """
    column_array = M[starting_row:, column] # starting from the specified row, get values along the specified column
    for i, val in enumerate(column_array):
        if not np.isclose(val, 0, atol=1e-5): # if not close to 0
            index = i + starting_row # is what we aim to find, get the index
            return index
    return -1 # if no non-zero values in the column


In [5]:
def swap_rows(M, row_index_1, row_index_2):
    """
    Auxiliary function to swap 2 rows given their indices
    """
    M = M.copy()
    M[row_index_1], M[row_index_2] = M[row_index_2], M[row_index_1]
    return M

In [6]:
def row_echelon_form(A, B):
    # Check singularity of matrices (infinite or no solutions)
    det_A = np.linalg.det(A)
    if np.isclose(det_A, 0):
        return "Singular system"

    # Make copies of the input to avoid modifying the original
    A, B = A.copy(), B.copy()

    # Convert matrices to float to prevent integer division
    A, B = A.astype('float64'), B.astype('float64')

    # Construct augmented matrix M = [A|B]
    M = augmented_matrix(A, B)

    # Transformation
    num_rows = len(A)
    for row in range(num_rows):
        pivot_candidate = M[row, row] # the first pivot candidate is in the diagonal

        if np.isclose(pivot_candidate, 0): # if the pivot candidate is 0
            first_non_zero_value_below_pivot_candidate = get_index_first_non_zero_value_from_column(M, row, row) # find the first non-zero entry in the column
            M = swap_rows(M, row, first_non_zero_value_below_pivot_candidate) # swap rows with non-zero entry in the column
        
        else: # is non-zero
            pivot = pivot_candidate # set as pivot of the row
        
        # Apply row reduction below the pivot
        M[row] = M[row] * 1 / pivot # divide to get pivot value to 1, normalize accordingly the current row
        
        for j in range(row + 1, num_rows): # for rows below the current row
            value_below_pivot = M[j, row]
            M[j] = M[j] - M[row] * value_below_pivot # subtract the above row to get the value below pivot to 0, change accordingly the whole row j
    return M

### Step 3: Backsubstitution


Backsubstitution initiates from the bottom and goes upward, aiming to convert every element above the pivot into zeros. The final result is a reduced row  echelon form matrix. The formula applied to rows:
$$\text{Row above} \rightarrow \text{Row above} - \text{value} \cdot \text{Row pivot}$$

After transforming to row echelon form as in step 2, the pivots are all 1, but there are non-zero and non-one values above the pivots. For example, consider the matrix:
$$
M = 
\begin{bmatrix} 
\phantom{-}1 & -1 & \phantom{-}\frac{1}{2} & | & \phantom{-}\frac{1}{2} \\
\phantom{-}0 & \phantom{-}1 & \phantom{-}1 & | & -1 \\
\phantom{-}0 & \phantom{-}0 & \phantom{-}1 & | & -1 
\end{bmatrix}
$$

Starting from bottom (row 2) to top (row 0):

- $R_2$:

- -  $R_1 = R_1 - 1 \cdot R_2 = \begin{bmatrix} 0 & 1 & 0 & | & 0 \end{bmatrix}$
- - $R_0 = R_0 - \frac{1}{2} \cdot R_2 = \begin{bmatrix} 1 & -1& 0 & | & 1 \end{bmatrix}$

The resulting matrix is then

$$
M = 
\begin{bmatrix} 
\phantom{-}1 & -1 & \phantom{-}0 & | & \phantom{-}1  \\
\phantom{-}0 & \phantom{-}1 & \phantom{-}0 & | & \phantom{-}0 \\
\phantom{-}0 & \phantom{-}0 & \phantom{-}1 & | & -1 
\end{bmatrix}
$$

Moving upwards to $R_1$:

- $R_1$:

- - $R_0 = R_0 - \left(-1 \cdot R_1 \right) = \begin{bmatrix} 1 & 0 & 0 & | & 1 \end{bmatrix}$

And the final matrix is

$$
M = 
\begin{bmatrix} 
\phantom{-}1 & \phantom{-}0 & \phantom{-}0 & | & \phantom{-}1  \\
\phantom{-}0 & \phantom{-}1 & \phantom{-}0 & | & \phantom{-}0 \\
\phantom{-}0 & \phantom{-}0 & \phantom{-}1 & | & -1
\end{bmatrix}
$$



In [9]:
def get_index_first_non_zero_value_from_row(M, row, augmented=False):
    """
    Auxiliary function to get the index of the first non-zero value from a specified row
    """
    M = M.copy()
    if augmented == True: # only get values from the original matrix, not constant column
        M = M[:, :-1] # every row, every column except the last column
    row_array = M[row]
    for i, val in enumerate(row_array):
        if np.isclose(val, 0, atol=1e-5):
            return i
    return -1
        

In [10]:
def backsubstitution(M):
    M = M.copy()
    num_rows = len(M)
    for row in reversed(range(num_rows)): # go bottom up 
        substitution_row = M[row] # fix the row to subtract the rows above from
        pivot = get_index_first_non_zero_value_from_row(M, row, augmented=True) # get index of the pivot of the row

        for j in range(row): # loop through all rows above the current row
            row_to_reduce = M[j]
            value = row_to_reduce[pivot] # the values above the pivot
            row_to_reduce = row_to_reduce - substitution_row * value
            M[j, :] = row_to_reduce[:]
    return M[:, -1] # return the constant column, solution of the reduced echelon form augmented matrix


### Step 4: Combine into one function

In [11]:
def gaussian_elimination(A, B):
    row_echelon_M = row_echelon_form(A, B)
    res = backsubstitution(row_echelon_M)
    return res

### Test

In [15]:
equations = """
3*x + 6*y + 6*w + 8*z = 1
5*x + 3*y + 6*w = -10
4*y - 5*w + 8*z = 8
4*w + 8*z = 9
"""

A = np.array([[3, 6, 6, 8], [5, 3, 6, 0], [4, -5, 0, 8], [0, 0, 4, 8]])
B = np.array([[1], [-10], [8], [9]])
gaussian_elimination(A, B)


array([ 46.97540984,   1.66666667, -49.58333333,   1.27459016])